<a href="https://colab.research.google.com/github/tar697/AI-ML-project/blob/main/Image_style_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio torch torchvision pillow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 49.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [7]:
# Installing all the tools and libraries
import torch                                         #deep learning framework
import torch.nn as nn                                # define and use neural network layers, like convolution or loss functions
import torchvision.models as models                  # To use pretrained models (like VGG19) from the torchvision library
import torchvision.transforms as transforms          # For image processing
from PIL import Image                                # open and process image files using the Python Imaging Library (PIL)
import gradio as gr                                  # Loads Gradio, a tool for creating web apps
import copy


In [9]:
# For checking if a GPU is available (via CUDA)
device = torch.device("cuda" if torch.cuda.is_available()
                                else "cpu")

In [10]:
# To prepare input images for the model
loader = transforms.Compose([                       #↓↓↓ Faster loading with smaller size ↓↓↓
    transforms.Resize((256, 256)),                  # Optimized for speed
    transforms.ToTensor()                           # Converts the image from PIL format to a PyTorch tensor
])
unloader = transforms.ToPILImage()                   # converts a tensor back into a displayable image

In [12]:
# Defines a function that takes a PIL image as input and prepares it for the model
def image_loader(image):
    image = loader(image).unsqueeze(0)
    return image.to(device, torch.float)

# To creates a reusable PyTorch module for content loss
class ContentLoss(nn.Module):
    def __init__(self, target):
        super().__init__()
        self.target = target.detach()
    def forward(self, x):
        self.loss = nn.functional.mse_loss(x, self.target)
        return x

# Computes the Gram matrix to represent style
def gram_matrix(x):
    b, c, h, w = x.size()
    features = x.view(b * c, h * w)
    G = torch.mm(features, features.t())
    return G.div(b * c * h * w)


class StyleLoss(nn.Module):
    def __init__(self, target_feature):
        super().__init__()
        self.target = gram_matrix(target_feature).detach()
    def forward(self, x):
        G = gram_matrix(x)
        self.loss = nn.functional.mse_loss(G, self.target)
        return x

def get_style_model_and_losses(cnn, style_img, content_img):
    cnn = copy.deepcopy(cnn)
    content_layers = ['conv_4']
    style_layers = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']

    content_losses = []
    style_losses = []

    model = nn.Sequential().to(device)
    i = 0

    for layer in cnn.children():
        if isinstance(layer, nn.Conv2d):
            i += 1
            name = f"conv_{i}"
        elif isinstance(layer, nn.ReLU):
            name = f"relu_{i}"
            layer = nn.ReLU(inplace=False)
        elif isinstance(layer, nn.MaxPool2d):
            name = f"pool_{i}"
        else:
            continue

        model.add_module(name, layer)

        if name in content_layers:
            target = model(content_img).detach()
            content_loss = ContentLoss(target)
            model.add_module(f"content_loss_{i}", content_loss)
            content_losses.append(content_loss)

        if name in style_layers:
            target = model(style_img).detach()
            style_loss = StyleLoss(target)
            model.add_module(f"style_loss_{i}", style_loss)
            style_losses.append(style_loss)

    for i in range(len(model) - 1, -1, -1):
        if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
            break
    model = model[:(i + 1)]

    return model, style_losses, content_losses

def run_style_transfer(cnn, content_img, style_img, num_steps=50, style_weight=1e6, content_weight=1):
    print("🚀 Starting style transfer...")
    input_img = content_img.clone()
    optimizer = torch.optim.LBFGS([input_img.requires_grad_()])
    model, style_losses, content_losses = get_style_model_and_losses(cnn, style_img, content_img)

    run = [0]
    while run[0] <= num_steps:
        def closure():
            optimizer.zero_grad()
            model(input_img)
            style_score = sum([sl.loss for sl in style_losses])
            content_score = sum([cl.loss for cl in content_losses])
            loss = style_weight * style_score + content_weight * content_score
            loss.backward()

            if run[0] % 10 == 0:
                print(f"Step {run[0]}/{num_steps} - Style Loss: {style_score:.4f}, Content Loss: {content_score:.4f}")
            run[0] += 1
            return loss
        optimizer.step(closure)

    print("✅ Style transfer complete.")
    return input_img.detach()

In [11]:
# To take a content image and a style image, and return a stylized image that blends them using deep learning.
def style_transfer_app(content_img, style_img):
    cnn = models.vgg19(pretrained=True).features.to(device).eval()  #Loads a pretrained VGG19 convolutional neural network
    content = image_loader(content_img)
    style = image_loader(style_img)
    output = run_style_transfer(cnn, content, style)
    result = unloader(output.squeeze(0).cpu())
    return result
# This creates a Gradio UI
interface = gr.Interface(
    fn=style_transfer_app,
    inputs=[
        gr.Image(type="pil", label="Upload Content Image"),
        gr.Image(type="pil", label="Upload Style Image")
    ],
    outputs=gr.Image(type="pil", label="Stylized Output"),
    title="🎨 AI Style Transfer (Optimized)",
    description="Fast style transfer with reduced image size and steps. Upload content and style images to generate art."
)

interface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://50b6b21b3337f988b9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
